In [1]:
!pip install -U abacusai

In [2]:
import collections
import threading
import time

from tqdm import tqdm
import numpy as np
import IPython.display

import abacusai

# Set up API client and load data

In [3]:
api_key = 'c5a0c0bca947417f86f789aad7797701'
server = 'https://abacus.ai'

In [4]:
api_client = abacusai.ApiClient(api_key=api_key, server=server)
api_client

In [5]:
feature_group = api_client.describe_feature_group('8b6f96aa')
data = feature_group.load_as_pandas()
data.head()

,vehicle_title,review_title,review
0,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,"my 4th previa, best van ever made!",1st 95 went over 300k before being totalled b...
1,1997 Toyota Previa Minivan LE 3dr Minivan,Mom's Taxi Babies Ride,Sold 86 Toyota Van 285K miles to be replaced ...
2,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,Best Minivan ever,My 1997 AWD Previa is the third one that I ha...
3,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,Final model year of the great Previa,"An amazing vehicle: mid-engine, supercharged,..."
4,2007 Toyota Avalon Sedan XLS 4dr Sedan (3.5L 6...,I'll drive this car until they take it away fr...,Bought this Avy in 2007 used with 1200 miles ...


# Get predictions from sentiment model

In [6]:
from abacusai import PredictionClient
client = PredictionClient()
client.get_sentiment(deployment_token='9bc6d5bad8f144aa92da6a5fe1019671', deployment_id='16b7a3a714', document="I love my car!")

{'admiration': 0.011122096734024428,
 'amusement': 0.0005833048933439076,
 'anger': 0.0008364225831216573,
 'annoyance': 0.0007938346446064794,
 'approval': 0.0022172060616474,
 'caring': 0.0006706955851472972,
 'confusion': 0.0004855435197345739,
 'curiosity': 0.0005260815380606611,
 'desire': 0.0004729935069821223,
 'disappointment': 0.0006606700039959563,
 'disapproval': 0.0006746487853362465,
 'disgust': 0.0003855836185036351,
 'embarrassment': 0.00019420315884523563,
 'excitement': 0.002207837238762706,
 'fear': 0.00014659781698434177,
 'gratitude': 0.0012260011991796153,
 'grief': 0.00011197871465645072,
 'joy': 0.011642383283344169,
 'love': 0.993821215471438,
 'nervousness': 0.00016667645628123717,
 'neutral': 0.0018604800906387063,
 'optimism': 0.0012090200608950248,
 'pride': 0.00016631956744411256,
 'realization': 0.000882531064327018,
 'relief': 5.5772003104732163e-05,
 'remorse': 7.353746443918921e-05,
 'sadness': 0.0005549830029762105,
 'surprise': 0.00029549331878569063}

In [7]:
sent_deployment_token = '9bc6d5bad8f144aa92da6a5fe1019671'
sent_deployment_id = '16b7a3a714'

In [8]:
tqdm._instances.clear()
sample_texts = data['review'][:100]
sentiments = [
    client.get_sentiment(
        deployment_token=sent_deployment_token,
        deployment_id=sent_deployment_id,
        document=text
    )
    for text in tqdm(sample_texts)
]

100%|██████████| 100/100 [00:49<00:00,  2.01it/s]


In [9]:
query = 'joy'
print(f'Top scoring texts for: "{query}"\n')
scores = [s[query] for s in sentiments]
arg_sort = np.argsort(-np.array(scores))
for i in arg_sort[:5]:
    print(scores[i])
    print(sample_texts[i])
    print('')

Top scoring texts for: "joy"

0.9765579227783318
 Sold 86 Toyota Van 285K miles to be replaced with 97 Previa with 1st baby coming in Jun. Perfect to carry 2nd kid in 99. Also carried Mom, Dad, Father-in-law to doctor's appt. Use mostly for reserve weekend duty. Did not change remote bat 6 yrs later. Did reg maint oil/filter chg every 3K. Used in business for 6 yrs as modified pick up truck by removing middle seat & folding 3rd row. Kids love this because they can lie down and sleep and watch tv. Our kids and parents in their 80s enjoyed trips to NY, VA, FL, LA, GA, TN, SC, NC, AL, TX, MA, CO. With this economy we are glad to have Toyotas in our family. Camry 02 replaced 88 Celica. Convinced five younger brothers to get Toyotas. They did.

0.9703472903251203
 I've had the XL model since Aug. '07 and have about 6500K ultra-enjoyable miles on it. I've had two Beemers and an Infinity, also enjoyed them very much, but the Avalon is special in that it's smooth, quiet, fairly ecnomical on ga

In [10]:
query = 'annoyance'
print(f'Top scoring texts for: "{query}"\n')
scores = [s[query] for s in sentiments]
arg_sort = np.argsort(-np.array(scores))
for i in arg_sort[:5]:
    print(scores[i])
    print(sample_texts[i])
    print('')

Top scoring texts for: "annoyance"

0.9876571410165871
 I bought my 2011 Avalon Limited new in Dec.2011. It is not the same quality of my two previous Avalons.Last one was 2002 with 176K miles and it ran smoother and quieter than the new one. Wind comes in window, I'm told it's normal due to mirrow (doesn't do it on passenger side). Rides rough, worse than my husbands F250 4x4 diesel. Lots and lots of road noise.Not a quiet car at all. Rear passenger window broke (fixed now), rear reclining seat broke (part ordered). Brakes feel mushy, Struts seem non-existent. Car really sat low when 2 adults and 2 teens with 4 bags packed (not heavy..trunk too small to pack much). Toyota dealer tells me everything is normal and up to Toyota specs. Bad Car!!

0.9837900807035953
 Overall our ‘13 Avalon is a compromise  in meeting our expectations for comfort, quasi-sportiness, and initial quality.  Best features include the space for 6’-4” legs and the visibility afforded to urban driving conditions.  

In [11]:
query = 'fear'
print(f'Top scoring texts for: "{query}"\n')
scores = [s[query] for s in sentiments]
arg_sort = np.argsort(-np.array(scores))
for i in arg_sort[:5]:
    print(scores[i])
    print(sample_texts[i])
    print('')

Top scoring texts for: "fear"

0.9927920479839639
 Would love 2009 Avalon except for one major problem.  At unexpected times after I brake the car begins to move forward.  This has happened to 3 different drivers.  I cannot stop this forward movement. It caused me to rear end another car at a band drive through.  Toyota dealers will not acknowledge problem since they cannot duplicate problem.  Car has 3000 miles on it and I am afraid to drive it. I have been told that I am the first person to report this in the U.S. I feel robbed. This is going to happen again.

0.9297390076755588
 I paid $36,000 for my 2009 Avalon. I will not drive it. It has 3000 miles on it.  On 8 occasions the car has lurched forward after brakes were applied. One accident occurred costing me $500 deduct. I was in the bank line when the brakes failed. Car started moving forward.  Brakes would not stop the car.  Toyota dealers say they cannot find a problem. Both my husband and brother have experienced same problem 

# Create a classification model

In [12]:
print('Creating project')
class_project = api_client.create_project('nlp_classification', use_case='NLP_CLASSIFICATION')

api_client.add_feature_group_to_project(
    feature_group_id=feature_group, project_id=class_project, feature_group_type='DOCUMENTS'
)

class_project.set_feature_mapping(
    feature_group_id=feature_group,
    feature_name='review',
    feature_mapping='DOCUMENT',
)

print('Training model')
class_model = class_project.train_model(
    name='classification_model_1',
    training_config={
        'zero_shot_hypotheses': [
            'This text is about car speed / acceleration / slowness',
            'This text is about car price / cost / value for money',
            'This text is about car build quality',
            'This text is about car seats',
        ]
    }
)

class_model = class_model.wait_for_full_automl()
print('Deploying model')
class_deployment = api_client.create_deployment(model_id=class_model)
class_deployment = class_deployment.wait_for_deployment()
class_deployment_token = class_project.create_deployment_token()
class_deployment_id = class_deployment.deployment_id

print('Model deployed')

Creating project
Training model
Deploying model
Model deployed


In [ ]:
print('Testing prediction')
client.get_classification(
    deployment_token=class_deployment_token,
    deployment_id=class_deployment_id,
    document="This car is a bargain."
)

sample_texts = data['review'][:100]
text_classifications = []

text_to_prediction = {}

print('Producing predictions')

# Produce predictions in the background so we can do some analysis straight away

def process_data(texts):
    for text in texts:
        text_to_prediction[text] = client.get_classification(
            deployment_token=class_deployment_token,
            deployment_id=class_deployment_id,
            document=text,
        )
        
thread = threading.Thread(target=process_data, args=(sample_texts,))
thread.start()

while len(text_to_prediction) < 20:
    IPython.display.clear_output(wait=True)
    print(f'Predictions produced so far: {len(text_to_prediction)}')
    time.sleep(1)
print('Example prediction:')
list(text_to_prediction.values())[0]

Predictions produced so far: 0


In [ ]:
predictions = []
for text in sample_texts:
    if text not in text_to_prediction:
        break
    else:
        predictions.append(text_to_prediction[text])
print(f'Predictions made so far: {len(predictions)}\n')
query = list(predictions[0].keys())[1]
print(f'Top scoring texts for: "{query}"\n')
scores = [s[query] for s in predictions]
arg_sort = np.argsort(-np.array(scores))
for i in arg_sort[:5]:
    print(f'Score = {scores[i]}')
    print(sample_texts[i])
    print('')

# Build a named entity recognition (NER) model

In [ ]:
ner_project = api_client.create_project('named_entity_recognition', use_case='NAMED_ENTITY_RECOGNITION')

api_client.add_feature_group_to_project(
    feature_group_id=feature_group, project_id=ner_project, feature_group_type='DOCUMENTS'
)

ner_project.set_feature_mapping(
    feature_group_id=feature_group,
    feature_name='review',
    feature_mapping='DOCUMENT',
)

ner_model = ner_project.train_model(
    name='ner_model_1',
    training_config={'NER_MODEL_TYPE': "pretrained uncased 43 categories"}
)

ner_model.wait_for_full_automl()

ner_deployment = api_client.create_deployment(model_id=ner_model)

ner_deployment = ner_deployment.wait_for_deployment()

ner_deployment_token = ner_project.create_deployment_token()

ner_deployment_id = ner_deployment.deployment_id
ner_deployment_token = ner_deployment_token

client.get_labels(deployment_token=ner_deployment_token, deployment_id=ner_deployment_id, query_data={"content":"I love my Toyota Camry!"})

In [ ]:
tqdm._instances.clear()
sample_texts = data['review'][:100]
annotations = [
    client.get_labels(deployment_token=ner_deployment_token, deployment_id=ner_deployment_id, query_data={'content': text})['annotations']
    for text in tqdm(sample_texts)
]

In [ ]:
label_counts = collections.Counter([anno['displayName'] for anno_list in annotations for anno in anno_list])
label_counts.most_common(10)

In [ ]:
product_counts = collections.Counter([anno['textExtraction']['textSegment']['phrase'].strip().lower()
                                      for anno_list in annotations for anno in anno_list if anno['displayName'] == 'product'])
product_counts.most_common(10)

# Combined analysis using multiple models

In [ ]:
# Filter using NER

filtered_texts = [
    text
    for text, anno_list in zip(sample_texts, annotations)
    if any([anno['displayName'] == 'product' and 'avalon' in anno['textExtraction']['textSegment']['phrase'].strip().lower()
           for anno in anno_list])
]
len(filtered_texts)

In [ ]:
# See top scores after the filter

predictions = []
for text in filtered_texts:
    if text not in text_to_prediction:
        break
    else:
        predictions.append(text_to_prediction[text])
query = list(predictions[0].keys())[1]
print(f'Top scoring texts for: "{query}"\n')
scores = [s[query] for s in predictions]
arg_sort = np.argsort(-np.array(scores))
for i in arg_sort[:5]:
    print(scores[i])
    print(filtered_texts[i])
    print('')